패키지 로드

In [1]:
import torch
import pandas as pd
import numpy as np
import csv
import requests
from hanspell import spell_checker

# 맞춤법 교정

- py-hanspell 설치 후 진행
- 파이썬 버전 3.10이상

In [33]:
from hanspell import spell_checker

In [3]:
data_kurly = pd.read_csv('C:/Users/NT550-048/Desktop/잇더/data/data_kurly_sentiment.csv', index_col = 0)

In [ ]:
data_kurly['ReviewText'] = data_kurly['ReviewText'].apply(lambda x:x.replace('&', ',') if '&' in x else x)

In [13]:
data_kurly['fixed_ReviewText'] = data_kurly['ReviewText'].apply(lambda x:spell_checker.check(x).checked)

500자이상 리뷰 처리
- 500자 이상은 hanspell 불가
- 500자 이상 리뷰 문장별로 나눠서 맞춤법 교정

In [ ]:
from kss import split_sentences

In [65]:
tokens = []
for i in range(len(data_kurly)):
    try:
        tokens.append(get_pos(data_kurly['fixed_ReviewText'][i]))
    except:
        print('ERROR:',i)
        tokens.append('ERROR')

ERROR: 1083
ERROR: 1198
ERROR: 3263
ERROR: 5034
ERROR: 5533
ERROR: 8168
ERROR: 8522
ERROR: 10039
ERROR: 10253
ERROR: 10290


In [25]:
# 500자 이상 리뷰는 hanspell 실행 불가
# 문장별로 쪼개서 hanspell 수행
nums = [1083, 1198, 3263, 5034, 5533, 8168, 8522, 10039, 10253, 10290]
for n in nums:
    temp_longreview_list = split_sentences(data_kurly['ReviewText'][n])
    longreview_checked = []
    for txt in temp_longreview_list:
        longreview_checked.append(spell_checker.check(txt).checked)
    data_kurly['fixed_ReviewText'][n] = '. '.join(longreview_checked)

C:\Users\NT550-048\AppData\Local\Temp\ipykernel_13372\482859583.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_kurly['fixed_ReviewText'][n] = '. '.join(longreview_checked)


# 토큰화

In [27]:
# 한국어 형태소 분석
from konlpy.tag import Okt, Hannanum, Kkma, Mecab, Komoran
# 형태소 벡터를 생성하기 위한 라이브러리
from sklearn.feature_extraction.text import CountVectorizer
# 형태소 벡터를 학습 벡터로 변환한다.
from sklearn.feature_extraction.text import TfidfTransformer

형태소 추출 함수
- stem : True면 원형 어간으로

In [28]:
# konlpy 라이브러리로 텍스트 데이터에서 형태소를 추출한다.
def get_pos (x) :
    tagger = Okt()
    #pos = tagger.pos(x)
    pos = tagger.pos(x, norm = True, stem = True)
    
    # 단어와 품사를 합쳐서 하나의 단어로 만들어준다.
    result = []
    
    # 형태소의 수만큼 반복한다.
    # 조사인 것과 명사인 것이 같을 수 있기 때문에 구분해준다.
    # 형태소 벡터를 만들때 추후 사용
    for a1 in pos :
        result.append(f'{a1[0]}/{a1[1]}')
    
    return result

토큰화 수행

In [97]:
data_kurly['tokens'] = data_kurly['fixed_ReviewText'].apply(lambda x:get_pos(x))

맞춤법 교정/토큰화 데이터 저장

In [103]:
data_kurly.to_csv("C:/Users/NT550-048/Desktop/잇더/data/data_kurly_tokens.csv")